In [52]:
import numpy as np
import pandas as pd

In [53]:
# get most important coeffs from lasso-logit-full
coeffs = np.loadtxt("../coef_lasso_logit_full.txt")
top_words = coeffs.argsort()[-10:][::-1]
print(coeffs[top_words])

[2.88317862 2.20474637 1.89043858 1.88126299 1.84982683 1.77917599
 1.74436485 1.67609757 1.64098876 1.55399143]


In [54]:
# look at which words correspond to the most important coeff
vocablist = pd.read_csv("../vocab10K.csv")
word_map = np.loadtxt("../i_keep_columns.txt")
vocab_indices = word_map[top_words] + 1
print(vocablist.loc[vocablist['index'].isin(vocab_indices)])

      index        word  female  i_pronoun  exclude      coef        ME  \
1106   1107         hot     NaN        NaN      NaN  1.849972  0.271002   
1530   1531   beautiful     NaN        NaN      NaN  1.637034  0.239809   
1696   1697  attractive     NaN        NaN      NaN  1.672941  0.245069   
3298   3299       marry     NaN        NaN      NaN  1.879690  0.275356   
4135   4136    pregnant     NaN        NaN      NaN  2.203002  0.322718   
5067   5068   pregnancy     NaN        NaN      NaN  1.734884  0.254143   
7146   7147    marrying     NaN        NaN      NaN  1.775510  0.260094   
7889   7890      breast     NaN        NaN      NaN  1.546361  0.226526   
8914   8915      hotter     NaN        NaN      NaN  2.882882  0.422313   
9137   9138        plow     NaN        NaN      NaN  1.892775  0.277273   

      nFemale  nMale  coef_pronoun  ME_pronoun  nFemale_pronoun  \
1106     3613   1053      1.220669    0.197421             1309   
1530     1419    610      0.925590    0.

In [55]:
# HELPER: dump all the gender coded words into a file
vocab10K=pd.read_csv("../vocab10K.csv")
male = vocab10K.loc[vocab10K['female'] == 0,:]
male = male['word'].tolist()
female = vocab10K.loc[vocab10K['female'] == 1,:]
female = female['word'].tolist()

In [65]:
# HELPER: label data
path = "../custom_data/all_tweets.csv"
#path = "../custom_data/economics_posts.csv"
data = pd.read_csv(path)
vocab = pd.read_csv("../vocab10K.csv")
num_entries = data.shape[0]
y = list()
male_set = set(male)
female_set = set(female)
nFemale = 0
nMale = 0
unambiguous_posts = list()
for i in range(num_entries):
    if i % 1000 == 0:
        print(i)
    post = str(data['text'][i]).lower()
    i_male, i_female = 0, 0
    for word in post.split():
        if word in male_set:
            i_male += 1
        elif word in female_set:
            i_female += 1
    if i_male > i_female:
        y.append(0)
        nMale += 1
        unambiguous_posts.append(post)
    if i_female > i_male:
        y.append(1)
        nFemale += 1
        unambiguous_posts.append(post)
print("nMale: ", nMale)
print("nFemale: ", nFemale)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
nMale:  1057
nFemale:  622


In [66]:
# Helper: make bow
vocab = pd.read_csv("../vocab10K.csv")
num_entries = len(unambiguous_posts)
X = np.zeros((num_entries, 10000))
vocab_list = vocab['word'].tolist()
vocab_set = set(vocab_list)
for i in range(num_entries):
    if i % 100 == 0:
        print(i)
    post = unambiguous_posts[i].split()
    for word in post:
        if word in vocab_set:
            X[i, vocab_list.index(word)] = 1

np.save("twitter_data", X)
np.save("twitter_labels", y)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [67]:
# get most important coeffs from lasso-reddit
coeffs = np.loadtxt("../coef_twitter.txt")
women_words = coeffs.argsort()[-15:][::-1]
men_words = coeffs.argsort()[:15]
# look at which words correspond to the most important coeff
vocablist = pd.read_csv("../vocab10K.csv")
word_map = np.loadtxt("../i_keep_columns.txt")
women_indices = word_map[women_words] + 1
men_indices = word_map[men_words] + 1
print("women: ")
print(vocablist.loc[vocablist['index'].isin(women_indices)]['word'])
print("men: ")
print(vocablist.loc[vocablist['index'].isin(men_indices)]['word'])

women: 
309         person
436      including
463          macro
480          black
486        support
667          share
877     colleagues
1275         drive
1344     interests
1463    undergrads
3512    bargaining
3642      speakers
4090            em
6426      murdered
7286     discusses
Name: word, dtype: object
men: 
207          theory
221           point
292          others
307          either
383             yet
966          modern
979           death
1196          types
1245       building
2226       presence
2324       struggle
3370       pipeline
4223    forthcoming
5927    fascinating
6580            der
Name: word, dtype: object
